In [1]:
import cv2

In [2]:
import numpy as np

In [3]:
from cv2 import resize, imwrite, imshow, boundingRect, countNonZero, cvtColor, drawContours, findContours, getStructuringElement, imread, morphologyEx, pyrDown, rectangle, threshold

In [4]:
# Function for Text Extraction
def process_rgb(rgb,n):
    hasText=0
    # Resize the image(different parameter for each document) so that the width of the text boxes are around 30 pixels
    rgb = resize(rgb,(800,550))
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY);
    
    # Create a 3*3 Eliptical Kernel for travesring the image. 
    morphKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,3))
    
    # Morphological Gradient is the difference between Dialtion and Erosion
    # The result will look like an outline of an object
    grad = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, morphKernel)
    
    # Thresholding the image using Binary and OSTU Threshold
    _, bw = cv2.threshold(grad, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # After getting the Binarized output,connect horizontally oriented regions since they are part of a single line.
    # In the function MORPH_RECT the values can be changed depending on the output desired. If we want to detect a sentence then
    # the value must be large. If we want to detect single words then the value must be small.
    morphKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
    connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, morphKernel)
    
    # Find all the contours after connecting the horizontally oriented regions so that they become part of one contour itself
    mask = np.zeros(bw.shape[:2], dtype="uint8");
    _,contours, hierarchy = cv2.findContours(connected, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    
    # We only need those contours where text is present
    j=0
    idx = 0
    while idx >= 0 :
        # Create Rectangles of Width=w and Height=h around the identified contours
        x,y,w,h = cv2.boundingRect(contours[idx]);
        
        # Create a Mask around a contour
        cv2.drawContours(mask, contours, idx, (255, 255, 255), cv2.FILLED);
        
        # Find the ratio of Non-Zero Regions inside the mask        
        r = cv2.contourArea(contours[idx])/(w*h)
        
        # If the ratio is greater than some constant, then there is text present inside the contour
        if(r > 0.05 and h > 10 and w > 50 and h < 80):
            cv2.rectangle(rgb, (x,y), (x+w,y+h), (0, 0, 0), 1)
            roi = rgb[y:y+h, x:x+w]
            imwrite("roi" + str(n)+str(idx)+".jpg", roi)
            j = j+1
            hasText = 1
        idx = hierarchy[0][idx][0]
     
    # Save the image
    imwrite(r"C:\Users\u293217\Downloads\\x"+str(n)+".jpg", rgb)

In [5]:
# Read the image
for i in range(1,11,1):
        img = imread(r"C:\Users\u293217\Downloads\\" + str(i) + ".jpg")
        process_rgb(img,i)